In [1]:
import serial
import time

## List ports in use

In [2]:
ls /dev/tty*

/dev/tty    /dev/tty23  /dev/tty39  /dev/tty54      /dev/ttyS0   /dev/ttyS24
/dev/tty0   /dev/tty24  /dev/tty4   /dev/tty55      /dev/ttyS1   /dev/ttyS25
/dev/tty1   /dev/tty25  /dev/tty40  /dev/tty56      /dev/ttyS10  /dev/ttyS26
/dev/tty10  /dev/tty26  /dev/tty41  /dev/tty57      /dev/ttyS11  /dev/ttyS27
/dev/tty11  /dev/tty27  /dev/tty42  /dev/tty58      /dev/ttyS12  /dev/ttyS28
/dev/tty12  /dev/tty28  /dev/tty43  /dev/tty59      /dev/ttyS13  /dev/ttyS29
/dev/tty13  /dev/tty29  /dev/tty44  /dev/tty6       /dev/ttyS14  /dev/ttyS3
/dev/tty14  /dev/tty3   /dev/tty45  /dev/tty60      /dev/ttyS15  /dev/ttyS30
/dev/tty15  /dev/tty30  /dev/tty46  /dev/tty61      /dev/ttyS16  /dev/ttyS31
/dev/tty16  /dev/tty31  /dev/tty47  /dev/tty62      /dev/ttyS17  /dev/ttyS4
/dev/tty17  /dev/tty32  /dev/tty48  /dev/tty63      /dev/ttyS18  /dev/ttyS5
/dev/tty18  /dev/tty33  /dev/tty49  /dev/tty7       /dev/ttyS19  /dev/ttyS6
/dev/tty19  /dev/tty34  /dev/tty5   /dev/tty8       /dev/ttyS2   /dev/ttyS7
/dev

## Connect to Arduino

In [8]:
ser = serial.Serial('/dev/ttyACM0', 56700) # Establish the connection on a specific port
ser.timeout = 2
ser.flushInput()

## Send Command to Arduino

In [3]:
def spincoat(rpm, duration): #duration = time it takes to get to target
    ser.flushInput()
    ser.write('r{0:d},{1:d}'.format(rpm, duration).encode())
#     msg=ser.readline()
#     print(msg.decode('utf-8'))

In [4]:
def setRPM(rpm): 
    ser.flushInput()
    ser.write('a{0:d}'.format(rpm).encode())
    time.sleep(.2)
    msg = ser.readline(); # read all characters in buffer
    print(msg.decode('utf-8'));

In [5]:
# ser.flushInput()
# ser.write('r0000,2000'.encode());

In [6]:
def checkRPM(): 
    ser.flushInput()
    ser.write('c'.encode())
    time.sleep(.2)
    msg = ser.readline(); # read all characters in buffer
    print(msg.decode('utf-8'));

## Coating Recipe

In [ ]:
def HTL():
    ser.flushImput
    spincoat(0, 0) # to ensure starting from off positon
    spincoat(4000, 1000)
    time.sleep(2)
    spincoat(0, 0)

In [ ]:
def PSK():
    spincoat(0, 0)
    spincoat(3000, 2000)
    time.sleep(2)
    spincoat(6000, 2000)
    time.sleep(2)
    spincoat(0, 0)

## Example set and hold specific RPM 

In [10]:
setRPM(0)

In [ ]:
checkRPM()

## Example running a coating recipe

In [391]:
PSK()

In [393]:
HTL()

In [11]:
import serial
import time

class SpinCoater:
	def __init__(self, port = '/dev/ttyACM2', baudrate=57600):
		#constants
		self.POLLINGRATE = 0.5 #query rate to arduino, in seconds
		# self.ACCELERATIONRANGE = (1,200) #rpm/s
		self.SPEEDRANGE = (1000, 9000) #rpm
		self.TERMINATOR = '\n'
		self.baudrate =57600
		self.connect(port =port) #where do i input baudrate?
		self.unlock()

	@property
	def rpm(self):
		self.write('c') #command to read rpm
		self.__rpm = float(self.__handle.readline().strip())
		return self.__rpm
	
	def connect(self, port, **kwargs):
		self.ser = serial.Serial()
		self.ser.baudrate = self.baudrate
		self.ser.port = self.port

		self.__handle = self.ser()
		self.__handle.open()


		# routine to initialize spincoater 
		# - cycle ESC power
		# - pwm to low, 2 seconds
		# - pwm to high, x seconds
		# finish

	def disconnect(self):
		self.__handle.close()

	def write(self, s):
		'''
		appends terminator and converts to bytes before sending message to arduino
		'''
		self.__handle.write(f'{s}{self.TERMINATOR}'.encode())
 	
	def lock(self):
		"""
		routine to lock rotor in registered position for sample transfer
		"""
		if self.locked:
			return

		self.write('z') # 
		time.sleep(2) #wait some time to ensure rotor has stopped and engaged with electromagnet
		self.write('i4') #send command to engage electromagnet
		time.sleep(2) #wait some time to ensure rotor has stopped and engaged with electromagnet
		self.locked = True

	def unlock(self):
		"""
		unlocks the rotor from registered position
		"""
		self.write('o4') #send command to disengage electromagnet
		time.sleep(2) #wait some time to ensure rotor has unlocked before attempting to rotate 
		self.locked = False

	def setspeed(self, speed): #acceleration = max(self.ACCELERATIONRANGE)):
		
		speed = int(speed) #arduino only takes integer inputs

		if self.locked:
			self.unlock()
		self.__handle.write('a{speed:d}') 

		#send command to arduino. assumes arduino responds to "s{rpm},{acceleration}\r'
		'''
		sends commands to arduino to set a target speed with a target acceleration

		speed - target angular velocity, in rpm
		acceleration - target angular acceleration, in rpm/second. always positive
		acceleration = int(acceleration) 
		#possible code to wait for confirmation response from arduino that speed was hit successfully
		'''

	def stop(self):
		"""
		stop rotation and locks the rotor in position
		"""
		self.write('z') # 
		self.lock()

	def recipe(self, recipe):
		"""
		executes a series of spin coating steps. A final "stop" step is inserted
		at the end to bring the rotor to a halt.

		recipe - nested list of steps in format:
			
			[
				[speed, acceleration, duration],
				[speed, acceleration, duration],
				...,
				[speed, acceleration, duration]
			]

			where speed = rpm, acceleration = rpm/s, duration = s

		"""
		# record = {
		# 	'time':[],
		# 	'rpm': []
		# 	}

		# start_time = time.time()
		# next_step_time = 0
		# time_elapsed = 0
		# for step in recipe:
		# 	speed = step[0]
		# 	acceleration = step[1]
		# 	duration = step[2]

		# 	while time_elapsed <= next_step_time:
		# 		record['time'].append(time_elapsed)
		# 		record['rpm'].append(self.rpm)
		# 		time.sleep(self.POLLINGRATE)

		# 	self.setspeed(speed, acceleration)
		# 	next_step_time += duration
		# self.stop()

		# return record





In [7]:
SpinCoater

__main__.SpinCoater

In [8]:
self.connect()

NameError: name 'self' is not defined

In [11]:
ser = serial.Serial()
ser.baudrate = 57600
ser.port = '/dev/ttyACM2'
ser

Serial<id=0x7f497c652210, open=False>(port='/dev/ttyACM2', baudrate=57600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)

In [12]:
serial.Serial(port='/dev/ttyACM2', baudrate=57600)

Serial<id=0x7f497c63c7d0, open=True>(port='/dev/ttyACM2', baudrate=57600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)